In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float


In [2]:
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    user_id = Column(Integer, primary_key=True)
    user_name = Column(String)
    openness_score = Column(Float)
    conscientiousness_score = Column(Float)
    extraversion_score = Column(Float)
    agreeableness_score = Column(Float)
    neuroticism_score = Column(Float)

class Book(Base):
    __tablename__ = 'books'
    book_id = Column(Integer, primary_key=True)
    title = Column(String)
    author = Column(String)
    genre = Column(String)
    description = Column(String)
    openness_match = Column(Float)
    conscientiousness_match = Column(Float)
    extraversion_match = Column(Float)
    agreeableness_match = Column(Float)
    neuroticism_match = Column(Float)

engine = create_engine('sqlite:///book_personality.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
db_session = Session()

C:\Users\Asus\AppData\Local\Temp\ipykernel_20196\2548969095.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
current_user = None
personality_results = None

personality_questions = [
    {"question_text": "I see myself as someone who is imaginative and creative", "trait": "openness"},
    {"question_text": "I prefer variety over routine", "trait": "openness"},
    {"question_text": "I am always prepared", "trait": "conscientiousness"},
    {"question_text": "I pay attention to details", "trait": "conscientiousness"},
    {"question_text": "I feel comfortable around people", "trait": "extraversion"},
    {"question_text": "I start conversations", "trait": "extraversion"},
    {"question_text": "I feel others' emotions", "trait": "agreeableness"},
    {"question_text": "I make people feel at ease", "trait": "agreeableness"},
    {"question_text": "I get irritated easily", "trait": "neuroticism"},
    {"question_text": "I often feel blue", "trait": "neuroticism"}
]

sample_books = [
    {
        "title": "The Art of Creative Thinking",
        "author": "John Adair",
        "genre": "Self-help",
        "description": "Unlock your creative potential with practical techniques.",
        "openness_match": 0.95,
        "conscientiousness_match": 0.4,
        "extraversion_match": 0.3,
        "agreeableness_match": 0.5,
        "neuroticism_match": 0.2
    },
    {
        "title": "Atomic Habits",
        "author": "James Clear",
        "genre": "Self-help",
        "description": "Build good habits and break bad ones with proven strategies.",
        "openness_match": 0.5,
        "conscientiousness_match": 0.9,
        "extraversion_match": 0.4,
        "agreeableness_match": 0.6,
        "neuroticism_match": 0.3
    },
    {
        "title": "How to Win Friends and Influence People",
        "author": "Dale Carnegie",
        "genre": "Self-help",
        "description": "Timeless advice for improving social skills.",
        "openness_match": 0.4,
        "conscientiousness_match": 0.5,
        "extraversion_match": 0.85,
        "agreeableness_match": 0.7,
        "neuroticism_match": 0.3
    },
    {
        "title": "The Book of Joy",
        "author": "Dalai Lama & Desmond Tutu",
        "genre": "Spirituality",
        "description": "Two spiritual leaders discuss finding joy in troubled times.",
        "openness_match": 0.6,
        "conscientiousness_match": 0.5,
        "extraversion_match": 0.4,
        "agreeableness_match": 0.9,
        "neuroticism_match": 0.2
    },
    {
        "title": "The Subtle Art of Not Giving a F*ck",
        "author": "Mark Manson",
        "genre": "Self-help",
        "description": "A counterintuitive approach to living a good life.",
        "openness_match": 0.7,
        "conscientiousness_match": 0.4,
        "extraversion_match": 0.5,
        "agreeableness_match": 0.3,
        "neuroticism_match": 0.8
    }
]

In [ ]:
if db_session.query(Book).count() == 0:
    for book_data in sample_books:
        book = Book(**book_data)
        db_session.add(book)
    db_session.commit()

def evaluate_personality(responses):
    """Calculate personality scores from responses"""
    scores = {
        'openness': 0,
        'conscientiousness': 0,
        'extraversion': 0,
        'agreeableness': 0,
        'neuroticism': 0
    }
    counts = {trait: 0 for trait in scores}
    
    for i, response in enumerate(responses):
        question = personality_questions[i]
        trait = question['trait']
        score = response if not question.get('reverse', False) else 6 - response
        scores[trait] += score
        counts[trait] += 1
    
    for trait in scores:
        if counts[trait] > 0:
            scores[trait] = round(scores[trait] / counts[trait], 2)
    
    return scores

def show_personality_assessment():
    global current_user, personality_results
    
    clear_output()
    
    if not current_user:
        display(widgets.HTML("<p style='color:red'>Please select a user first</p>"))
        return main_ui()
    
    display(widgets.HTML("<h2>Personality Assessment</h2>"))
    
    question_sliders = []
    for question in personality_questions:
        slider = widgets.IntSlider(
            min=1, max=5, step=1, value=3,
            description=question["question_text"],
            style={'description_width': 'initial'}
        )
        question_sliders.append(slider)
        display(slider)
    
    def submit_assessment(b):
        global personality_results
        
        responses = [slider.value for slider in question_sliders]
        personality_results = evaluate_personality(responses)
        
        if current_user:
            user = db_session.query(User).filter_by(user_id=current_user).first()
            user.openness_score = personality_results['openness']
            user.conscientiousness_score = personality_results['conscientiousness']
            user.extraversion_score = personality_results['extraversion']
            user.agreeableness_score = personality_results['agreeableness']
            user.neuroticism_score = personality_results['neuroticism']
            db_session.commit()
        
        clear_output()
        display(widgets.HTML("<h3>Assessment Results</h3>"))
        display(pd.DataFrame.from_dict(personality_results, orient='index', columns=['Score']))
        display(widgets.HTML("<p>Assessment saved successfully!</p>"))
        
        button_box = widgets.HBox()
        back_button = widgets.Button(description="Back to Main Menu")
        back_button.on_click(lambda b: main_ui())
        
        recommend_button = widgets.Button(
            description="Get Personalized Recommendations", 
            style={'button_color': '#2ecc71'}
        )
        recommend_button.on_click(lambda b: show_personality_recommendations())
        
        button_box.children = [back_button, recommend_button]
        display(button_box)
    
    submit_button = widgets.Button(description="Submit Assessment")
    submit_button.on_click(submit_assessment)
    display(submit_button)

def calculate_personality_match(user_scores, book):
    """Calculate how well a book matches the user's personality"""
    weights = {
        'openness': 0.25,
        'conscientiousness': 0.2,
        'extraversion': 0.2,
        'agreeableness': 0.2,
        'neuroticism': 0.15
    }
    
    total_score = 0
    match_details = {}
    
    for trait in user_scores:
        user_normalized = (user_scores[trait] - 1) / 4  
        book_normalized = getattr(book, f"{trait}_match")

        trait_match = 1 - abs(user_normalized - book_normalized)
        weighted_match = trait_match * weights[trait]
        total_score += weighted_match
        match_details[trait] = trait_match
    
    total_score = round(total_score * 100, 1)
    match_details = {k: round(v * 100, 1) for k, v in match_details.items()}
    
    return total_score, match_details

def show_personality_recommendations():
    clear_output()
    
    if not current_user:
        display(widgets.HTML("<p style='color:red'>Please select a user first</p>"))
        return main_ui()
    
    user = db_session.query(User).filter_by(user_id=current_user).first()
    
    if not user.openness_score:
        display(widgets.HTML("<p style='color:red'>Please complete the personality assessment first</p>"))
        return main_ui()
    
    user_scores = {
        'openness': user.openness_score,
        'conscientiousness': user.conscientiousness_score,
        'extraversion': user.extraversion_score,
        'agreeableness': user.agreeableness_score,
        'neuroticism': user.neuroticism_score
    }

    books = db_session.query(Book).all()
    book_matches = []
    
    for book in books:
        match_score, match_details = calculate_personality_match(user_scores, book)
        book_matches.append({
            'book': book,
            'match_score': match_score,
            'match_details': match_details
        })
    
    book_matches.sort(key=lambda x: x['match_score'], reverse=True)
    
    display(widgets.HTML(
        f"""<div style='background:#f8f9fa; padding:15px; border-radius:10px; margin-bottom:20px;'>
            <h2 style='color:#2c3e50;'>📚 Personalized Recommendations for {user.user_name}</h2>
            <div style='display:flex; justify-content:space-between; flex-wrap:wrap;'>
                <div style='margin:5px;'><b>Openness:</b> {user_scores['openness']}/5</div>
                <div style='margin:5px;'><b>Conscientiousness:</b> {user_scores['conscientiousness']}/5</div>
                <div style='margin:5px;'><b>Extraversion:</b> {user_scores['extraversion']}/5</div>
                <div style='margin:5px;'><b>Agreeableness:</b> {user_scores['agreeableness']}/5</div>
                <div style='margin:5px;'><b>Neuroticism:</b> {user_scores['neuroticism']}/5</div>
            </div>
        </div>"""
    ))
    
    for i, match in enumerate(book_matches[:5], 1):
        book = match['book']
        
        progress_bars = []
        for trait, score in match['match_details'].items():
            progress_bars.append(f"""
            <div style='margin-bottom:5px;'>
                <div style='display:flex; justify-content:space-between; font-size:12px;'>
                    <span>{trait.capitalize()}</span>
                    <span>{score}%</span>
                </div>
                <progress value='{score}' max='100' style='width:100%; height:8px;'></progress>
            </div>
            """)
        
        progress_bars_html = "".join(progress_bars)
        
        display(widgets.HTML(
            f"""<div style='border:1px solid #e0e0e0; border-radius:8px; padding:15px; margin-bottom:20px; 
                box-shadow:0 2px 4px rgba(0,0,0,0.05);'>
                <div style='display:flex; justify-content:space-between; align-items:center;'>
                    <h3 style='margin-top:0; color:#2980b9;'>#{i} {book.title}</h3>
                    <div style='background:#27ae60; color:white; padding:5px 10px; border-radius:4px;'>
                        <b>{match['match_score']}% Match</b>
                    </div>
                </div>
                <p><b>Author:</b> {book.author} | <b>Genre:</b> {book.genre}</p>
                <p style='color:#555;'>{book.description}</p>
                <div style='background:#f5f5f5; padding:10px; border-radius:6px; margin-top:10px;'>
                    <h4 style='margin-top:0; margin-bottom:10px; font-size:14px;'>Personality Traits Match:</h4>
                    {progress_bars_html}
                </div>
            </div>"""
        ))
    
    back_button = widgets.Button(description="Back to Main Menu", 
                               style={'button_color': '#3498db', 'font_weight': 'bold'})
    back_button.on_click(lambda b: main_ui())
    display(back_button)

def main_ui():
    global current_user
    
    if db_session.query(User).count() == 0:
        sample_users = ["Alex", "Jamie", "Taylor"]
        for name in sample_users:
            user = User(user_name=name)
            db_session.add(user)
        db_session.commit()
    
    user_dropdown = widgets.Dropdown(
        options=[(user.user_name, user.user_id) for user in db_session.query(User).all()],
        description='Select User:',
        style={'description_width': 'initial'}
    )
    
    def on_user_change(change):
        global current_user
        current_user = change['new']
    
    user_dropdown.observe(on_user_change, names='value')
    
    new_user_text = widgets.Text(description='New User Name:', style={'description_width': 'initial'})
    new_user_button = widgets.Button(description='Register New User')
    
    def register_new_user(b):
        if new_user_text.value:
            new_user = User(user_name=new_user_text.value)
            db_session.add(new_user)
            db_session.commit()
            user_dropdown.options = [(user.user_name, user.user_id) for user in db_session.query(User).all()]
            new_user_text.value = ''
    
    new_user_button.on_click(register_new_user)
    
    assessment_button = widgets.Button(
        description="Take Personality Assessment",
        style={'button_color': '#e67e22', 'font_weight': 'bold'}
    )
    assessment_button.on_click(lambda b: show_personality_assessment())
    
    display(widgets.VBox([
        widgets.HTML("<h1 style='color:#2c3e50; text-align:center;'>BookMatch Personality Recommendation</h1>"),
        widgets.HTML("<p style='text-align:center; color:#555;'>Discover books that match your personality traits</p>"),
        widgets.HBox([user_dropdown, new_user_text, new_user_button]),
        assessment_button
    ]))

main_ui()

HTML(value="<div style='background:#f8f9fa; padding:15px; border-radius:10px; margin-bottom:20px;'>\n         …

HTML(value="<div style='border:1px solid #e0e0e0; border-radius:8px; padding:15px; margin-bottom:20px; \n     …

HTML(value="<div style='border:1px solid #e0e0e0; border-radius:8px; padding:15px; margin-bottom:20px; \n     …

HTML(value="<div style='border:1px solid #e0e0e0; border-radius:8px; padding:15px; margin-bottom:20px; \n     …

HTML(value="<div style='border:1px solid #e0e0e0; border-radius:8px; padding:15px; margin-bottom:20px; \n     …

HTML(value="<div style='border:1px solid #e0e0e0; border-radius:8px; padding:15px; margin-bottom:20px; \n     …

Button(description='Back to Main Menu', style=ButtonStyle(button_color='#3498db', font_weight='bold'))